In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
gpu_id = '0'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
dataset_name = 'milan'
sub_set_name = 'call'
series_type = 'hourly'

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def preprocess(data, std=True):
    '''
    Function to do preprocess for input data
    inputs:
        + data: pandas.DataFrame
    outputs:
        + preprocessed_data: pandas.DataFrame
    '''
    if std:
        scaler = StandardScaler()
        temp = scaler.fit_transform(data)
    else:
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(data)
    processed_data = pd.DataFrame(temp, index=data.index, columns = data.columns)
    # processed_data = processed_data.to_numpy()
    return processed_data

In [6]:
file_path = f"./dataset/{dataset_name}/{dataset_name}_{sub_set_name}_{series_type}.csv"
raw_data = pd.read_csv(file_path)
raw_data = raw_data.pivot(index='startTime', columns='gridID', values=sub_set_name)
raw_data = raw_data.fillna(0)
raw_data = preprocess(raw_data, std=False)
raw_data.head()

gridID,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.056744,0.056868,0.056997,0.056358,0.055267,0.056997,0.056997,0.056997,0.056997,0.052012,...,0.071669,0.094064,0.132937,0.119432,0.067270,0.031713,0.032608,0.032041,0.024616,0.023914
2013-11-01 01:00:00,0.021376,0.021109,0.020834,0.022201,0.021617,0.020834,0.020834,0.020834,0.020834,0.016131,...,0.074835,0.073968,0.060506,0.043087,0.025463,0.012429,0.011351,0.011405,0.013007,0.015169
2013-11-01 02:00:00,0.009106,0.009065,0.009022,0.009234,0.009305,0.009022,0.009022,0.009022,0.009022,0.009141,...,0.019759,0.034425,0.064591,0.061299,0.031238,0.007821,0.008347,0.008025,0.006499,0.008660
2013-11-01 03:00:00,0.001552,0.001546,0.001540,0.001569,0.001852,0.001540,0.001540,0.001540,0.001540,0.002994,...,0.016774,0.022286,0.026397,0.012527,0.005052,0.002354,0.003822,0.003479,0.000364,0.001038
2013-11-01 04:00:00,0.001750,0.001647,0.001540,0.002070,0.002279,0.001540,0.001540,0.001540,0.001540,0.002005,...,0.014927,0.011260,0.002684,0.002901,0.002525,0.002312,0.002263,0.002273,0.002942,0.003292


In [7]:
# prompt: write pandas to csv

import pandas as pd

# Save the DataFrame to a CSV file
raw_data.rename(columns={'startTime': 'date'}, inplace=True)
raw_data.to_csv(f"./dataset/{dataset_name}/{sub_set_name}.csv", index_label='date')
raw_data.head()

gridID,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.056744,0.056868,0.056997,0.056358,0.055267,0.056997,0.056997,0.056997,0.056997,0.052012,...,0.071669,0.094064,0.132937,0.119432,0.067270,0.031713,0.032608,0.032041,0.024616,0.023914
2013-11-01 01:00:00,0.021376,0.021109,0.020834,0.022201,0.021617,0.020834,0.020834,0.020834,0.020834,0.016131,...,0.074835,0.073968,0.060506,0.043087,0.025463,0.012429,0.011351,0.011405,0.013007,0.015169
2013-11-01 02:00:00,0.009106,0.009065,0.009022,0.009234,0.009305,0.009022,0.009022,0.009022,0.009022,0.009141,...,0.019759,0.034425,0.064591,0.061299,0.031238,0.007821,0.008347,0.008025,0.006499,0.008660
2013-11-01 03:00:00,0.001552,0.001546,0.001540,0.001569,0.001852,0.001540,0.001540,0.001540,0.001540,0.002994,...,0.016774,0.022286,0.026397,0.012527,0.005052,0.002354,0.003822,0.003479,0.000364,0.001038
2013-11-01 04:00:00,0.001750,0.001647,0.001540,0.002070,0.002279,0.001540,0.001540,0.001540,0.001540,0.002005,...,0.014927,0.011260,0.002684,0.002901,0.002525,0.002312,0.002263,0.002273,0.002942,0.003292


In [8]:
# Download the dataset
# download_elec_dataset()

# # Clean dataset
# elec_data = clean_elec()

# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Std_normalization = 1
# if Std_normalization:
#     scaler = StandardScaler()
#     temp = scaler.fit_transform(elec_data)
#     norm_means = scaler.mean_
#     norm_std = scaler.scale_
# else:
#     scaler = MinMaxScaler()
#     temp = scaler.fit_transform(elec_data)

# elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(raw_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path='./dataset/milan/'
args.data_path='call.csv'
args.data_path

'call.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data[0][0].shape

(96, 10000)

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1000, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1000, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
      

# Setup WANDB

In [14]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.2
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Milan dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Milan Call dataset",
    # Hyperparams
    config={
        "dataset": "Milan Call",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 10000,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [15]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 cost time: 4.116959571838379
Epoch: 2 cost time: 3.045100450515747
Epoch: 3 cost time: 3.2812418937683105
Epoch: 4 cost time: 3.4994986057281494
Epoch: 5 cost time: 3.349930763244629
Epoch: 1 cost time: 3.5722365379333496
Epoch: 2 cost time: 3.202925682067871
Epoch: 3 cost time: 3.258208990097046
Epoch: 4 cost time: 3.4547953605651855
Epoch: 5 cost time: 4.19439959526062


Progress:   2%|▏         | 1/50 [00:52<42:57, 52.59s/it]

Epoch: 1 cost time: 3.3483195304870605
Epoch: 2 cost time: 2.9153664112091064
Epoch: 3 cost time: 2.5463497638702393
Epoch: 4 cost time: 2.7634458541870117
Epoch: 5 cost time: 2.7667489051818848
Epoch: 1 cost time: 2.7629284858703613
Epoch: 2 cost time: 2.6164495944976807
Epoch: 3 cost time: 2.6933438777923584
Epoch: 4 cost time: 3.351234197616577
Epoch: 5 cost time: 3.120293140411377


Progress:   4%|▍         | 2/50 [01:39<39:10, 48.96s/it]

...

Progress:  96%|█████████▌| 48/50 [36:05<01:30, 45.09s/it]

Epoch: 1 cost time: 2.611452102661133
Epoch: 2 cost time: 2.7414143085479736
Epoch: 3 cost time: 2.756295919418335
Epoch: 4 cost time: 2.6683170795440674
Epoch: 5 cost time: 2.6029977798461914
Epoch: 1 cost time: 2.615856409072876
Epoch: 2 cost time: 2.6213197708129883
Epoch: 3 cost time: 2.6658236980438232
Epoch: 4 cost time: 2.5820019245147705
Epoch: 5 cost time: 3.092862606048584


Progress:  98%|█████████▊| 49/50 [36:51<00:45, 45.34s/it]

Epoch: 1 cost time: 2.706667900085449
Epoch: 2 cost time: 2.7491159439086914
Epoch: 3 cost time: 2.7062649726867676
Epoch: 4 cost time: 2.707118272781372
Epoch: 5 cost time: 2.689405679702759
Epoch: 1 cost time: 2.799125909805298
Epoch: 2 cost time: 2.666731834411621
Epoch: 3 cost time: 2.7930688858032227
Epoch: 4 cost time: 3.049820899963379
Epoch: 5 cost time: 3.2678730487823486


Progress: 100%|██████████| 50/50 [37:36<00:00, 45.13s/it]
